![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/logo_MCV_W.png)

# **Keras - Transfer Learning with Cats vs Dogs**

---

In this lesson, we learn how to setup data generators to load our own dataset and train a classifier using Keras. 
1. Understand trainable layers of a Neural Network
2. Setting up our data
3. Building our Model for Transfer Learning
4. Perform Fine Tuning

In [1]:
# Import of libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random


## **Trainable Layers**

Layers & models have **three** weight attributes:

- `weights` is the list of all weights variables of the layer.
- `trainable_weights` is the list of those that are meant to be updated (via gradient
 descent) to minimize the loss during training.
- `non_trainable_weights` is the list of those that aren't meant to be trained.
 Typically they are updated by the model during the forward pass.

**Example: the `Dense` layer has 2 trainable weights (kernel & bias)**

In [ ]:
# layer = keras.layers.Dense(4)

# # Create the weights using layer.build
# layer.build((None, 2))  

# print(f'Number of weights: {len(layer.weights)}')
# print(f'Number of trainable_weights: {len(layer.trainable_weights)}')
# print(f'Number of non_trainable_weights: {len(layer.non_trainable_weights)}')

All layers are trainable with the exception of **BatchNormalization**. It uses non-trainable weights to keep track of the mean and variance of its inputs during training. 

**Layers & models** also feature a boolean attribute `trainable`. 

Its value can be changed by setting `layer.trainable` to `False` moves all the layer's weights from trainable to non-trainable.  

This is called **"freezing"** the layer: the state of a frozen layer won't
be updated during training (either when training with `fit()` or when training with
 any custom loop that relies on `trainable_weights` to apply gradient updates).

### **Example: setting `trainable` to `False`**

In [ ]:
# # Make a model with 2 layers
# layer1 = keras.layers.Dense(3, activation="relu")
# layer2 = keras.layers.Dense(3, activation="sigmoid")
# model = keras.Sequential([keras.Input(shape=(3,)), layer1, layer2])

# # Freeze the first layer
# layer1.trainable = False

# # Keep a copy of the weights of layer1 for later reference
# initial_layer1_weights_values = layer1.get_weights()

# # Train the model
# model.compile(optimizer="adam", loss="mse")
# model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

In [ ]:
# # Check that the weights of layer1 have not changed during training
# final_layer1_weights_values = layer1.get_weights()

# if initial_layer1_weights_values[0].all() == final_layer1_weights_values[0].all():
#   print('Weights unchanged')

# if initial_layer1_weights_values[1].all() == final_layer1_weights_values[1].all():
#   print('Weights unchanged')

**Note**: **`.trianable` is Recursive**, meaning that on a model or on any layer that has sublayers, all children layers become non-trainable as well.

## **Implementing Transfer Learning**

![](https://github.com/rajeevratan84/ModernComputerVision/blob/main/Screenshot%202021-05-11%20at%2011.49.01%20pm.png?raw=true)

## **Transfer-learning workflow**

1. We instantiate a **base model and load pre-trained weighs** into it.
2. **Freeze** all layers in the base model by setting `trainable = False`.
3. Create a **new model on top** of the output of one (or several) layers from the base
 model.
4. Train your new model on your new dataset.

### **Step 1. Load a base model with pre-trained weights (ImageNet)**

In [ ]:
# import tensorflow_datasets as tfds

# tfds.disable_progress_bar()

# train_ds, validation_ds, test_ds = tfds.load(
#     "cats_vs_dogs",
#     # Reserve 10% for validation and 10% for test
#     split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
#     as_supervised=True,  # Include labels
# )

# print(f'Number of training samples: {tf.data.experimental.cardinality(train_ds)}')
# print(f'Number of validation samples: {tf.data.experimental.cardinality(validation_ds)}')
# print(f'Number of test samples: {tf.data.experimental.cardinality(test_ds)}')

## **Standardize Our Data**

- Standardize to a fixed image size. We pick 150x150.
- Normalize pixel values between -1 and 1. We'll do this using a `Normalization` layer as
 part of the model itself.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import os
from scipy.stats import zscore

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.applications import VGG16, imagenet_utils
# from keras.callbacks import EarlyStopping
from keras.callbacks import EarlyStopping, ModelCheckpoint




In [3]:
met_control = pd.read_csv("../../Met/met_rr_control_named.csv")
met_control

,NSFTV_ID,Treatment,hexanoic acid,alanine,valine,urea,ethanolamine,leucine,glycerol,nicotinic acid,...,adenosine,trehalose,maltose,sophorose,melibiose,isomaltose,galactinol,phosphoric acid,Sucrose,Raffinose
0,NSFTV_1,Control,0.073463,0.099932,0.004720,1.853639,-0.269819,-0.055739,0.510914,0.391473,...,0.046634,-0.450524,-0.308289,-0.063850,0.423776,0.314369,0.158955,-0.140715,0.402484,-1.440777
1,NSFTV_102,Control,-0.015095,0.209659,0.127870,-0.101918,-0.437334,0.239351,0.055266,-0.028396,...,0.121284,-0.065042,0.368929,-0.008045,0.399713,-0.264498,-0.061229,-0.235017,-0.741653,1.070465
2,NSFTV_103,Control,0.160103,0.633219,0.151433,-0.387669,0.734534,0.257577,0.296091,0.659027,...,0.361614,-0.095921,1.005878,0.057135,0.015705,-0.009189,0.044330,0.834154,0.303034,-0.603933
3,NSFTV_104,Control,-0.013302,1.181448,2.265878,-0.359049,2.272740,2.541394,1.046026,0.542261,...,0.331713,-0.479455,1.581037,0.250587,-0.221493,2.584905,0.217831,1.466190,-0.346015,-2.386452
4,NSFTV_105,Control,0.328212,0.459086,0.923531,-0.247979,1.909945,0.371542,0.246153,-0.041397,...,0.674103,0.312100,0.409481,0.196645,-0.313199,0.029189,0.038495,1.258600,-0.338108,-0.278003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,NSFTV_90,Control,0.154189,-0.032636,-0.143583,0.443690,-0.013267,-0.008876,-0.090771,-0.303796,...,-0.009423,-0.107175,-0.492381,0.162906,0.192797,-0.192028,-0.061853,0.285540,-0.293308,1.110043
158,NSFTV_92,Control,-0.028981,-0.115577,0.092292,0.029275,0.301546,0.119252,-0.139018,-0.114555,...,-0.014101,0.000017,0.187861,0.185697,-0.062153,0.097698,-0.018249,-1.020564,0.129552,0.339379
159,NSFTV_93,Control,-0.005575,-0.058799,0.191081,0.850514,0.222220,0.368702,0.316836,-0.092219,...,-0.028387,0.096424,-0.680553,0.072835,-0.321447,-0.169982,-0.083892,-0.118708,-0.248154,-0.093230
160,NSFTV_96,Control,0.153888,0.111901,-0.248869,-0.651881,0.206074,-0.088784,0.612490,-0.194678,...,0.152969,-0.355506,-0.008856,-0.335311,-0.224398,-0.036093,-0.095945,-0.003359,-0.093124,-1.008269


In [5]:
met_control["glyceric acid"]

0     -0.002920
1     -0.027671
2      0.128017
3      0.155692
4      0.071120
         ...   
157   -0.008483
158   -0.010560
159   -0.026614
160    0.009631
161    0.013490
Name: glyceric acid, Length: 162, dtype: float64

In [ ]:
labels = zscore(np.array(met_control.glycerol)) #normalization
# labels = np.array(met_control.glycerol)
# print(labels)
plt.hist(labels)
lines = np.array(met_control.NSFTV_ID)
# print(lines)

dff = pd.DataFrame({
        "labels": labels,
        "NSFTV_ID": lines
    })

dff.head()

In [ ]:
dataset_path = "../../SNPimg/dataset/"
chr_path = os.listdir("../../SNPimg/dataset/NSFTV_1/")
base_df = dff.assign(dataset_path=dataset_path)
base_df.head()

In [ ]:
image_pathss = [base_df["dataset_path"] + base_df["NSFTV_ID"] + "/" + chr_path[i] for i in range(12)]  ####### here we can include multiple chromosome setting.
# print(image_paths)
print(np.array(image_pathss).shape)

In [ ]:
image_pathss[1]

In [ ]:
image_total = []

# loop over each batch

for image_paths in image_pathss:
  # print(image_paths)
  batch_size = 162
  # image_features_chr = []
  image_labels = []
  # image_paths = image_pathss[0]
  for i in range(0, len(image_paths)//batch_size):
    # print(i)
    batch_paths = image_paths[i:i + batch_size]
    batch_labels = dff["labels"][i:i + batch_size]
    batch_images = []
    for image_path in batch_paths:
      image = load_img(image_path, target_size = (224, 224)) #image is still in PIL format. Need to convert into np.array
      image = img_to_array(image)
      # We expand the dimensions and then subtract the mean RGB pixel intensity of ImageNet
      image = np.expand_dims(image, axis=0) # add one dim in axis=0
      image = imagenet_utils.preprocess_input(image) #normalization based on ImageNet.
      batch_images.append(image)
    
    batch_images = np.vstack(batch_images) #looks like no need to expand in the before lines.
    # features = model.predict(batch_images, batch_size = batch_size)
    # print(features.shape)
    # features = np.reshape(features,(-1, features.shape[1]*features.shape[2]*features.shape[3]))
    # print(features.shape)
    # # store our features and corresponding labels
    # image_features_chr.append(features)
    image_labels.append(batch_labels)

  image_total.append(batch_images)

# image_features.append(np.array(image_features_chr))  

In [30]:
i = 1
i = str(i)
cvv = pd.read_csv("../../Met/CrossValidation/cv_"+i+"/met_cv_"+i+".csv")


In [ ]:
train_index = cvv.query("Treatment=='Control' and set == 'train'").index
test_index = cvv.query("Treatment=='Control' and set == 'test'").index

train_labels = batch_labels[train_index]
test_labels = batch_labels[test_index]

Train_image = []
Test_image = []
for chrr in range(0,12):
    train_images = image_total[chrr][train_index,:,:,:]
    test_images = image_total[chrr][test_index,:,:,:]
    
    Train_image.append(train_images)
    Test_image.append(test_images)


In [ ]:
size = (150, 150)

def newdataset_ge(inputs, targets, size):
    inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)  # Adjust dtype as needed
    targets = tf.convert_to_tensor(targets, dtype=tf.float32)  # Adjust dtype as needed

    # Create a dataset using from_tensor_slices
    new_dataset = tf.data.Dataset.from_tensor_slices(({"inputs1": tf.image.resize(inputs[0], size=size), 
                                                       "inputs2": tf.image.resize(inputs[1], size=size), 
                                                       "inputs3":tf.image.resize(inputs[2], size=size),
                                                       "inputs4": tf.image.resize(inputs[3], size=size), 
                                                       "inputs5": tf.image.resize(inputs[4], size=size), 
                                                       "inputs6":tf.image.resize(inputs[5], size=size),
                                                       "inputs7": tf.image.resize(inputs[6], size=size), 
                                                       "inputs8": tf.image.resize(inputs[7], size=size), 
                                                       "inputs9":tf.image.resize(inputs[8], size=size),
                                                       "inputs10": tf.image.resize(inputs[9], size=size), 
                                                       "inputs11": tf.image.resize(inputs[10], size=size), 
                                                       "inputs12":tf.image.resize(inputs[11], size=size)}, 
                                                       targets))
    return new_dataset

In [ ]:
train_ds = newdataset_ge(Train_image, train_labels, size)
test_ds = newdataset_ge(Test_image, test_labels, size)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i, data in enumerate(train_ds.take(9)):
    # print(label)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(data[0]["inputs1"])
    # plt.title('Cat' if int(label) == 0 else 'Dog')
    plt.title(i)
    plt.axis("off")

In [ ]:
# size = (150, 150)

# train_ds = train_ds.map(lambda x, y: (tf.image.resize(x["inputs1"], size), y))
# validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
# test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

We'll batch the data and use caching & prefetching to optimize loading speed.

In [ ]:
batch_size = 8

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
# validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

### **Introduce some random data augmentation**

In [ ]:
# from tensorflow import keras
# from tensorflow.keras import layers

# data_augmentation = keras.Sequential(
#     [
#         layers.experimental.preprocessing.RandomFlip("horizontal"),
#         layers.experimental.preprocessing.RandomRotation(0.1),
#     ]
# )

#### **Visualize our Data Augmentations**

In [ ]:
# import numpy as np

# for images, labels in train_ds.take(1):
#     plt.figure(figsize=(10, 10))
#     first_image = images[0]
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         augmented_image = data_augmentation(
#             tf.expand_dims(first_image, 0), training=True
#         )
#         plt.imshow(augmented_image[0].numpy().astype("int32"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

## **3. Building our model**

Now let's built a model that follows the blueprint we've explained earlier.

Note that:

- We add a `Normalization` layer to scale input values (initially in the `[0, 255]`
 range) to the `[-1, 1]` range.
- We add a `Dropout` layer before the classification layer, for regularization.
- We make sure to pass `training=False` when calling the base model, so that
it runs in inference mode, so that batchnorm statistics don't get updated
even after we unfreeze the base model for fine-tuning.

- We'll be using the **Xception Model** as our base.

In [ ]:
base_model_vgg = keras.applications.VGG16(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model

base_model_vgg.trainable = False

# base_model_rs = keras.applications.ResNet(
#     weights="imagenet",  # Load weights pre-trained on ImageNet.
#     input_shape=(150, 150, 3),
#     include_top=False,
# )  # Do not include the ImageNet classifier at the top.

# # Freeze the base_model
# base_model_rs.trainable = True


In [ ]:

# Create new model on top
inputs1 = keras.Input(shape=(150, 150, 3), name="inputs1")
inputs2 = keras.Input(shape=(150, 150, 3), name="inputs2")
inputs3 = keras.Input(shape=(150, 150, 3), name="inputs3")
inputs4 = keras.Input(shape=(150, 150, 3), name="inputs4")
inputs5 = keras.Input(shape=(150, 150, 3), name="inputs5")
inputs6 = keras.Input(shape=(150, 150, 3), name="inputs6")
inputs7 = keras.Input(shape=(150, 150, 3), name="inputs7")
inputs8 = keras.Input(shape=(150, 150, 3), name="inputs8")
inputs9 = keras.Input(shape=(150, 150, 3), name="inputs9")
inputs10 = keras.Input(shape=(150, 150, 3), name="inputs10")
inputs11 = keras.Input(shape=(150, 150, 3), name="inputs11")
inputs12 = keras.Input(shape=(150, 150, 3), name="inputs12")

def multi_branch_build(inputs, base_model):

    # inputs = keras.Input(shape=(150, 150, 3))
    # x = data_augmentation(inputs)  # Apply random data augmentation

    x = inputs
    # Pre-trained Xception weights requires that input be scaled
    # from (0, 255) to a range of (-1., +1.), the rescaling layer
    # outputs: `(inputs * scale) + offset`
    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    x = scale_layer(x)

    # The base model contains batchnorm layers. We want to keep them in inference mode
    # when we unfreeze the base model for fine-tuning, so we make sure that the
    # base_model is running in inference mode here.
    x = base_model(x, training=False)
    # x=keras.layers.GlobalAveragePooling2D()(x)
  # Regularize with dropout
    return x

x1 = multi_branch_build(inputs = inputs1, base_model=base_model_vgg)
x2 = multi_branch_build(inputs = inputs2, base_model=base_model_vgg)
x3 = multi_branch_build(inputs = inputs3, base_model=base_model_vgg)
x4 = multi_branch_build(inputs = inputs4, base_model=base_model_vgg)
x5 = multi_branch_build(inputs = inputs5, base_model=base_model_vgg)
x6 = multi_branch_build(inputs = inputs6, base_model=base_model_vgg)
x7 = multi_branch_build(inputs = inputs7, base_model=base_model_vgg)
x8 = multi_branch_build(inputs = inputs8, base_model=base_model_vgg)
x9 = multi_branch_build(inputs = inputs9, base_model=base_model_vgg)
x10 = multi_branch_build(inputs = inputs10, base_model=base_model_vgg)
x11 = multi_branch_build(inputs = inputs11, base_model=base_model_vgg)
x12 = multi_branch_build(inputs = inputs12, base_model=base_model_vgg)

y = keras.layers.concatenate([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12])
y = keras.layers.GlobalAveragePooling2D()(y)
y = keras.layers.Dropout(0.2)(y)
y = keras.layers.Dense(256, activation='relu')(y)
y = keras.layers.Dropout(0.2)(y)
y = keras.layers.Dense(128, activation='relu')(y)
outputs = keras.layers.Dense(1)(y)
model = keras.Model([inputs1, inputs2, inputs3, inputs4, inputs5, inputs6, inputs7, inputs8, inputs9, inputs10, inputs11, inputs12], outputs)

model.summary()

In [ ]:
# Use conda to install these two packages!!!

# !conda install graphviz
# !conda install pydot

In [ ]:
tf.keras.utils.plot_model(model)


## **Now let's Train our Top Layer**

Note from the above summary that we only have 2,049 trainable paramaters.

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)


model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanSquaredError()],
    # batch_size=32
)

es = EarlyStopping(monitor='val_loss', patience=100)
epochs = 500

model.fit(train_ds, 
        epochs=epochs, 
        validation_data=test_ds,
        callbacks = [es])

In [ ]:
plt.plot(model.history.history['loss'], label='loss')
plt.plot(model.history.history['val_loss'], label = 'val_loss')

plt.xlabel('Epoch')
plt.ylabel('LOSS')
plt.legend(loc='upper right')
plt.show()

In [ ]:
model.evaluate(test_ds)

In [ ]:
y_pred = model.predict(test_ds)

from scipy.stats import pearsonr
correlation, _ = pearsonr(np.array(y_pred.reshape(32)), np.array(test_labels))
print(correlation)